# Рекомендация тарифов

Проектная работа

Описание проекта
Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».
В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.
Постройте модель с максимально большим значением accuracy. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте accuracy на тестовой выборке самостоятельно.

In [1]:
import math

import pandas as pd
import seaborn as sns
import scipy.stats as st
import numpy as np
import matplotlib.pyplot as plt

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sys import getsizeof


try:
    from IPython.core.display_functions import display
except ModuleNotFoundError:
    pass


In [2]:
%config InlineBackend.figure_formats = ['svg']

In [3]:
# seaborn style via MATPLOTLIBRC
custom_params = {'figure.figsize': (10, 6),
                'figure.facecolor': '#232425',
                'figure.dpi': 240,

                'legend.frameon': False,
                'legend.borderpad': 1.4,
                'legend.labelspacing': 0.7,
                'legend.handlelength': 0.7,
                'legend.handleheight': 0.7,

                'axes.facecolor': '#232425',
                'axes.labelcolor': '#EEEEEE',
                'axes.labelpad': 17,
                'axes.spines.left': False,
                'axes.spines.bottom': False,
                'axes.spines.right': False,
                'axes.spines.top': False,
                'axes.grid': False,

                'contour.linewidth': 0.0,

                'xtick.color': '#AAAAAA',
                'ytick.color': '#AAAAAA',
                'xtick.bottom': True,
                'xtick.top': False,
                'ytick.left': True,
                'ytick.right': False,

                'text.color': '#EEEEEE',
                'font.sans-serif': [
                    'Helvetica',
                    'Verdana',
                    'sans-serif'
                    ]
                }
sns.set_theme(style='darkgrid', rc=custom_params)
sns.set_context("notebook", font_scale=1.37)

In [4]:
random_seed = 108108108

## Откройте файл с данными и изучите его.

In [5]:
try:
    accu_df = pd.read_csv('./datasets/users_behavior.csv')
except FileNotFoundError:
    accu_df = pd.read_csv('https://code.s3.yandex.net/datasets/users_behavior.csv')
    print('FYI csv loaded by url')

In [6]:
display(
    accu_df.sample(5)
)

,calls,minutes,messages,mb_used,is_ultra
1414,44.0,287.93,9.0,27093.07,0
2598,19.0,145.13,6.0,5181.60,1
1958,35.0,224.79,1.0,8017.23,0
731,47.0,273.60,60.0,24264.53,0
1912,36.0,282.91,55.0,22405.02,0


In [7]:
display(
    accu_df.describe()
)

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


In [8]:
display(
    accu_df.info()
)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


None

In [9]:
accu_df = accu_df.astype(
    {
        'calls': 'uint8',
        'minutes': 'float32',
        'messages': 'uint8',
        'mb_used': 'float32',
        'is_ultra': 'bool',
    }
)

In [10]:
print(f'Размер набора данных после замены типов данных:',
    round((getsizeof(accu_df) / 1024), 2)
    , 'KB'
)

Размер набора данных после замены типов данных: 34.67 KB


In [11]:
print(
    'Поьзователей Ultra:',
    len(accu_df.loc[accu_df['is_ultra'] == True].index)
)
print(
    'Поьзователей Smart:',
    len(accu_df.loc[accu_df['is_ultra'] == False].index)
)

Поьзователей Ultra: 985
Поьзователей Smart: 2229


## Разделите исходные данные на обучающую, валидационную и тестовую выборки.

In [12]:
features = accu_df.drop(['is_ultra'], axis=1)
target = accu_df['is_ultra']


> Спрятанной тестовой выборки нет. Значит, данные нужно разбить на три части: обучающую, валидационную и тестовую. Размеры тестового и валидационного наборов обычно равны. Исходные данные разбивают в соотношении 3:1:1.


Используем стандартные пропорции для деления набора данных:
- обучающая выборка 60%
- валидационная выборка 20%
- тестовая выборка 20%


In [13]:
features_train, features_valid,\
target_train, target_valid = train_test_split(
    features, target,
    test_size=0.2,
    shuffle = True,
    random_state=random_seed
)

features_train, features_test,\
target_train, target_test = train_test_split(
    features_train,
    target_train,
    test_size=0.25,
    random_state=random_seed
)

In [14]:
print('x_train', features_train.shape)
print('x_valid', features_valid.shape)
print('x_test', features_test.shape)
print()
print('y_train', target_train.shape)
print('y_valid', target_valid.shape)
print('y_test', target_test.shape)


x_train (1928, 4)
x_valid (643, 4)
x_test (643, 4)

y_train (1928,)
y_valid (643,)
y_test (643,)


Соотношения выборок похожи на правду 👏

## Исследуйте качество разных моделей, меняя гиперпараметры.

### DecisionTreeClassifier

Подберём параметр `max_depth`

In [15]:
best_accu = 0
best_depth = 0
for depth in range(1, 11):
    tree_model = DecisionTreeClassifier(
        random_state=random_seed,
        max_depth=depth
    )

    tree_model.fit(features_train, target_train)
    tree_predictions = tree_model.predict(features_valid)
    tree_accuracy = accuracy_score(target_valid, tree_predictions)
    print('max_depth =', depth, ':', tree_accuracy)
    if tree_accuracy > best_accu:
        best_accu = tree_accuracy
        best_depth = depth

print('best depth:', best_depth)

max_depth = 1 : 0.7060653188180405
max_depth = 2 : 0.749611197511664
max_depth = 3 : 0.7729393468118196
max_depth = 4 : 0.7558320373250389
max_depth = 5 : 0.7698289269051322
max_depth = 6 : 0.7589424572317263
max_depth = 7 : 0.76049766718507
max_depth = 8 : 0.7620528771384136
max_depth = 9 : 0.7558320373250389
max_depth = 10 : 0.7558320373250389
best depth: 3


### RandomForestClassifier

In [16]:
best_est = 0
best_depth = 0
best_min_sam = 0
best_result = 0
for est in range(1, 11):
    for depth in range(1, 11):
        for min_sam in range(2, 7):
            forest_model = RandomForestClassifier(
                random_state=random_seed,
                n_estimators=est,
                max_depth=depth,
                min_samples_split=min_sam,
            )

            forest_model.fit(features_train, target_train)
            forest_result = forest_model.score(features_valid, target_valid)

            if forest_result > best_result:
                best_est = est
                best_depth = depth
                best_min_sam = min_sam
                best_result = forest_result
                print('n_estimators:', best_est)
                print('max_depth:', depth)
                print('min_samples_split:', min_sam)
                print(round(best_result, 4))

print('Итог:')
print('n_estimators:', best_est)
print('max_depth:', depth)
print('min_samples_split:', min_sam)
print(round(best_result, 4))

n_estimators: 1
max_depth: 1
min_samples_split: 2
0.7045
n_estimators: 1
max_depth: 2
min_samples_split: 2
0.7465
n_estimators: 1
max_depth: 3
min_samples_split: 2
0.7652
n_estimators: 1
max_depth: 6
min_samples_split: 2
0.7698
n_estimators: 1
max_depth: 7
min_samples_split: 2
0.7729
n_estimators: 1
max_depth: 8
min_samples_split: 2
0.7792
n_estimators: 3
max_depth: 6
min_samples_split: 2
0.7869
n_estimators: 7
max_depth: 8
min_samples_split: 3
0.7885
n_estimators: 9
max_depth: 10
min_samples_split: 2
0.79
n_estimators: 9
max_depth: 10
min_samples_split: 3
0.7932
n_estimators: 10
max_depth: 10
min_samples_split: 3
0.7978
Итог:
n_estimators: 10
max_depth: 10
min_samples_split: 6
0.7978


Проверим на тестовых данных:

In [17]:
forest_model = RandomForestClassifier(
    random_state=random_seed,
    n_estimators=9,
    min_samples_split=4,
    # max_features=5,
)
forest_model.fit(features_train, target_train)

forest_predict = forest_model.predict(features_test)
forest_accuracy = accuracy_score(target_test, forest_predict)

print('Точность RandomForestClassifier:', forest_accuracy)

Точность RandomForestClassifier: 0.8040435458786936


### LogisticRegression

In [18]:
log_reg_model = LogisticRegression(
    random_state=random_seed,
    solver='lbfgs',
    max_iter=3000,
    # fit_intercept=False,
)

log_reg_model.fit(features_train, target_train)

log_reg_model.score(features_valid, target_valid)
print(log_reg_model.score(features_valid, target_valid))
# dump(model, 'model_9_1.joblib')

0.6609642301710731


### Support Vector Classifier

In [19]:
svc_model = SVC()
svc_model.fit(features_train, target_train)

svc_prediction = svc_model.predict(features_valid)

print(accuracy_score(svc_prediction, target_valid))

# But Confusion Matrix and Classification Report give more details about performance
print(confusion_matrix(svc_prediction, target_valid))


0.7013996889580093
[[410 180]
 [ 12  41]]


### K-Nearest Neighbors Classifier

In [20]:
for n in range(1, 10):
    knn_model = KNeighborsClassifier(n_neighbors=n)
    knn_model.fit(features_train, target_train)
    knn_prediction = knn_model.predict(features_valid)

    print(accuracy_score(knn_prediction, target_valid))
    # print(classification_report(knn_prediction, target_valid))

0.6640746500777605
0.6951788491446346
0.6998444790046656
0.7153965785381027
0.7200622083981337
0.7309486780715396
0.7309486780715396
0.7278382581648523
0.7356143079315708


### LinearDiscriminantAnalysis

In [23]:
best_solver = None
lda_best_score = 0
lda_grid_solver=['svd', 'lsqr', 'eigen']


for solver in lda_grid_solver:
    lin_discr_an_model = LinearDiscriminantAnalysis(solver=solver)
    lin_discr_an_model.fit(features_train, target_train)
    lin_discr_an_prediction = lin_discr_an_model.predict(features_valid)
    lda_accu = accuracy_score(lin_discr_an_prediction, target_valid)
    if lda_accu > lda_best_score:
        lda_best_score = lda_accu
        best_solver = solver
    print(lda_accu)

print('best score', lda_best_score,
'best solver', solver)

0.7216174183514774
0.7216174183514774
0.7216174183514774
best score 0.7216174183514774 best solver eigen


### Кратко напишите выводы исследования.

## Проверьте качество модели на тестовой выборке.

## Дополнительное задание: проверьте модели на вменяемость.

## Описание данных

Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:

`сalls` — количество звонков,
`minutes` — суммарная длительность звонков в минутах,
`messages` — количество sms-сообщений,
`mb_used` — израсходованный интернет-трафик в Мб,
`is_ultra` — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

Как будут проверять мой проект?
Мы подготовили критерии оценки проекта, которыми руководствуются ревьюеры. Прежде чем приступить к решению кейса, внимательно их изучите.
На что обращают внимание ревьюеры, проверяя проект:

- Как вы изучаете данные после загрузки?
- Корректно ли разделяете данные на выборки?
- Как выбираете размеры выборок?
- Правильно ли вы оцениваете качество моделей в исследовании?
- Какие модели и гиперпараметры вы используете?
- Какие выводы об исследовании делаете?
- Правильно ли тестируете модели?
- Насколько высокое значение accuracy получаете?
- Соблюдаете структуру проекта и поддерживаете аккуратность кода?